# Statistical Analysis of Topological Indices

## Topological Indices
we use the following indices
- wiener index
- randic index
- generalized randic index
- harmonic index
- atom bond connectivity index
- first zagreb index
- second zagreb index

In [ ]:
# we need to read the dataset from the pickle file
import pickle

with open('dataset.pickle', 'rb') as f:
    dataset = pickle.load(f)

dataset.head()

In [ ]:
# we use grinpy to calculate the topological indices for each graph

# 70 min without wiener index
# wiener is not fast enough for ran and hypercubes, we skip it for now
import grinpy as gp
import tqdm

def get_topological_indices(G):
    ''' Create a dictionary with the topological indices of a graph G.'''
    topological_indices = {}
    topological_indices['wiener_index'] = gp.wiener_index(G)
    topological_indices['randic_index'] = gp.randic_index(G)
    topological_indices['generalized_randic_index'] = gp.generalized_randic_index(G, 2)
    topological_indices['harmonic_index'] = gp.harmonic_index(G)
    topological_indices['atom_bond_connectivity_index'] = gp.atom_bond_connectivity_index(G)
    topological_indices['first_zagreb_index'] = gp.first_zagreb_index(G)
    topological_indices['second_zagreb_index'] = gp.second_zagreb_index(G)

    return topological_indices

def topological_indices_all_graphs(dataSet, subtypes_to_skip = []):
    ''' Create a dictionary with the topological indices of all graphs in the dataset.'''
    topological_indices = {}

    for subType in dataSet:

        if subType in subtypes_to_skip:
            continue

        topological_indices[subType] = {}
        l = len(dataSet[subType])
        for (i, graph) in tqdm(enumerate(dataSet[subType])):
            topological_indices[subType][graph] = get_topological_indices(dataSet[subType][graph])
    return topological_indices


subtypes_to_skip = []

topological_indices_all_graphs = topological_indices_all_graphs(dataset, subtypes_to_skip)

print("done")